In [6]:
### Part2 (ii)
import numpy as np
import time
### Basic parameters
r = 0.0325
T = 2.0
S1_0 = 11.08
S2_0 = 73.4
sigma1 = 0.6039
sigma2 = 0.3481
rho12 = 0.5456
paths_list = [10000,300000]

### Generate correlated normal random numbers
def generate_correlated_normal(M,N,rho):
    x1 = np.random.normal(0,1,size=(M,N))
    x2 = np.random.normal(0,1,size=(M,N))
    eps1 = x1
    eps2 = rho * x1 + x2 * np.sqrt(1-rho**2)
    return eps1,eps2

##
def monte_carlo_part_ii(paths):
    start_time = time.perf_counter()
    delta_ts = [0.5, 0.5, 1.0]  # Three time steps
    num_steps = len(delta_ts)
    
    # Generate relevant random numbers for each time step
    eps1, eps2 = generate_correlated_normal(M=paths, N=num_steps, rho=rho12)
    
    # Initialize the price
    S1_prev = np.full(paths, S1_0)
    S2_prev = np.full(paths, S2_0)
    B = np.zeros((paths, 3))  
    
    # Calculate prices step by step over time
    for i in range(num_steps):
        delta_t = delta_ts[i]
        e1, e2 = eps1[:, i], eps2[:, i]
        
        # Precise discretization
        drift1 = (r - 0.5 * sigma1**2) * delta_t
        diffusion1 = sigma1 * e1 * np.sqrt(delta_t)
        S1_current = S1_prev * np.exp(drift1 + diffusion1)
        
        drift2 = (r - 0.5 * sigma2**2) * delta_t
        diffusion2 = sigma2 * e2 * np.sqrt(delta_t)
        S2_current = S2_prev * np.exp(drift2 + diffusion2)
        
        # Record the B value
        B[:, i] = np.minimum(S1_current/S1_0, S2_current/S2_0)
        
        # Update the price from the previous moment
        S1_prev, S2_prev = S1_current, S2_current
    
    # Calculate the payment and option price
    A = np.mean(B, axis=1)
    payoffs = np.maximum(100.0 - A * 100.0, 0.0)
    option_price = np.exp(-r * T) * np.mean(payoffs)
    compute_time = time.perf_counter() - start_time
    
    return option_price, compute_time

if __name__ == "__main__":
    np.random.seed(42)  
    print("\n=== Part II(ii)：N=3 ===")
    for paths in paths_list:
        price, comp_time = monte_carlo_part_ii(paths)
        print(f"paths：{paths:>6d} | Option price：{price:.4f}% | Computation time：{comp_time:.4f}second")




=== Part II(ii)：N=3 ===
paths： 10000 | Option price：23.2183% | Computation time：0.0032second
paths：300000 | Option price：23.1444% | Computation time：0.1013second
